<a href="https://colab.research.google.com/github/LuckyYidan/IAI/blob/main/exp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Packages

In [ ]:
%%time
!pip install optuna
!pip install kaleido

# # !pip install -r /content/drive/MyDrive/tab-ddpm/requirements.txt
!pip install zero
!pip install icecream
!pip install category_encoders
!pip install tomli_w
!pip install rtdl
!pip install catboost
!pip install libzero==0.0.8
!pip install -U skorch
!pip install dython==0.6.4.post1
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.0 MB/s eta 0:00:00
     ━━━

# Import Packages


In [ ]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append('/content/drive/MyDrive/ThesisFinal')
from self import *
from self import Vime, Scarf
from supervised import *
from generative import *
from generative import CTGAN
from utils import *
from utils import FeatureEngineering

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from importlib import reload
supervised = reload(sys.modules["supervised"])
from supervised import *



utils = reload(sys.modules["utils"])
from utils import *
self = reload(sys.modules["self"])
from self import *
from self import Vime, Scarf

In [ ]:
SMOTETuner = reload(sys.modules["generative"]).SMOTETuner

# Import Wandb

In [ ]:
import wandb
wandb.init(
    project='experiment1',
    config={
        'industry_label':{
            '绿色环保产业':0,
            '生物产业':1,
            '新材料产业':2,
            '高端装备制造产业':3,
            '新一代信息技术产业':4,
            '新能源产业':5,
            '航空航天产业':6,
            '海洋工程装备产业':3,
            '新能源汽车产业':5

        },
        'categorical_encoding':'CountVectorizer',
        'numerical imputation':'median',
        'data normalization':'quantile',
        'dimension reduction':None,
        'feature enginerring':'xgboost + random forest'


    }
)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Prepare Dir

In [ ]:
_EXP1_DATA_DIR = '/content/drive/MyDrive/ThesisFinal/experiment1/experiment1'
_EDA_DIR = '/content/drive/MyDrive/ThesisFinal/EDA'
_DICT_DIR = '/content/drive/MyDrive/ThesisFinal/dict'
_PARENT_DIR = '/content/drive/MyDrive/ThesisFinal/experiment1'
_RESULTS_DIR = create_dir(_PARENT_DIR,'results')
_EXP1_EDA_DIR = create_dir(_EDA_DIR,'exp1')
_SUPERVISED_RESULTS_DIR = create_dir(_RESULTS_DIR,'supervised')
_SUPERVISED_XGB_RESULTS_DIR = create_dir(_SUPERVISED_RESULTS_DIR,'xgb_features')
_SUPERVISED_RF_RESULTS_DIR = create_dir(_SUPERVISED_RESULTS_DIR,'rf_features')
_SUPERVISED_COMBINE_RESULTS_DIR = create_dir(_SUPERVISED_RESULTS_DIR,'keep_features')
_SUPERVISED_SMALL_RESULTS_DIR = create_dir(_SUPERVISED_RESULTS_DIR,'small_features')
_SUPERVISED_FINETUNING = create_dir(_RESULTS_DIR,'fine_tuning')
_SELF_RESULTS_DIR = create_dir(_RESULTS_DIR,'self')
_SELF_VIME_RESULTS_DIR = create_dir(_SELF_RESULTS_DIR,'VIME')

_SELF_SCARF_RESULTS_DIR = create_dir(_SELF_RESULTS_DIR,'SCARF')
_GEN_SMOTE_DIR = create_dir(_EXP1_DATA_DIR,'smote_fake')
_GEN_CTGANPLUS_DIR = create_dir(_EXP1_DATA_DIR,'ctgan+')
_GEN_SMOTE_RESULT_DIR = create_dir(_RESULTS_DIR,'smote')
_GEN_CTGANPLUS_RESULT_DIR = create_dir(_RESULTS_DIR,'ctgan+')

/content/drive/MyDrive/ThesisFinal/experiment1/results
/content/drive/MyDrive/ThesisFinal/EDA/exp1
/content/drive/MyDrive/ThesisFinal/experiment1/results/supervised
/content/drive/MyDrive/ThesisFinal/experiment1/results/supervised/xgb_features
/content/drive/MyDrive/ThesisFinal/experiment1/results/supervised/rf_features
/content/drive/MyDrive/ThesisFinal/experiment1/results/supervised/keep_features
/content/drive/MyDrive/ThesisFinal/experiment1/results/supervised/small_features
/content/drive/MyDrive/ThesisFinal/experiment1/results/fine_tuning
/content/drive/MyDrive/ThesisFinal/experiment1/results/self
/content/drive/MyDrive/ThesisFinal/experiment1/results/self/VIME
/content/drive/MyDrive/ThesisFinal/experiment1/results/self/SCARF
/content/drive/MyDrive/ThesisFinal/experiment1/experiment1/smote_fake
/content/drive/MyDrive/ThesisFinal/experiment1/experiment1/ctgan+
/content/drive/MyDrive/ThesisFinal/experiment1/results/smote
/content/drive/MyDrive/ThesisFinal/experiment1/results/ctgan+


In [ ]:
_SUPERVISED_ALL_RESULTS_DIR = create_dir(_RESULTS_DIR,'all_features')

/content/drive/MyDrive/ThesisFinal/experiment1/results/all_features


# Load Data

In [ ]:
%%time
X_train = load_data(_EXP1_DATA_DIR,'X_train')
X_valid = load_data(_EXP1_DATA_DIR,'X_valid')
X_test = load_data(_EXP1_DATA_DIR,'X_test')
y_train = load_data(_EXP1_DATA_DIR,'y_train')
y_test = load_data(_EXP1_DATA_DIR,'y_test')
y_valid = load_data(_EXP1_DATA_DIR,'y_valid')
X_unlabel = load_data(_EXP1_DATA_DIR,'X_unlabel')
df = load_data(_EXP1_DATA_DIR,'industry_label')

CPU times: user 7.02 s, sys: 677 ms, total: 7.7 s
Wall time: 11.2 s


In [ ]:
X_unlabel.shape

(4994, 5127)

In [ ]:
X_train.shape

(1525, 5127)

In [ ]:
target='industry'

print('train:')
print(' features:',X_train.shape)
print(' target',y_train.shape)

print('valid:')
print(' features:',X_valid.shape)
print(' target',y_valid.shape)

print('test:')
print(' features:',X_test.shape)
print(' target',y_test.shape)

print('unlbel:')
print(' features:',X_unlabel.shape)
print(' target:',None)

train:
 features: (1525, 5127)
 target (1525, 1)
valid:
 features: (170, 5127)
 target (170, 1)
test:
 features: (727, 5127)
 target (727, 1)
unlbel:
 features: (4994, 5127)
 target: None


In [ ]:
target_eda = EDA(data = y_train,
                 target=target,
                 output_path = _EXP1_EDA_DIR,
                 logx = False,
                 logy = False)
target_eda.plot_hist(file_name='trainset',
                     subtitle = 'Source: experiment 1 - train',
                     maintitle = 'Target ')

In [ ]:
target_eda = EDA(data = y_valid,
                 target=target,
                 output_path = _EXP1_EDA_DIR,
                 logx = False,
                 logy = False)
target_eda.plot_hist(file_name='validation_set',
                     subtitle = 'Source: experiment 1 - validation',
                     maintitle = 'Target ')

In [ ]:
target_eda = EDA(data = y_test,
                 target=target,
                 output_path = _EXP1_EDA_DIR,
                 logx = False,
                 logy = False)
target_eda.plot_hist(file_name='test_set',
                     subtitle = 'Source: experiment 1 - test',
                     maintitle = 'Target ')

# Normalization

In [ ]:
num_feas = load_json(_DICT_DIR,'feas_num_sample')['numerical_features']

In [ ]:
num_feas.remove('prized_company')
num_feas.remove('env_label')

In [ ]:
normalized_cols = list(set(num_feas) - set(['patent_syxx_presv_total_3yr', 'patent_wgx_pledge_total_3yr', 'patent_presv_cnt_total_3yr', 'patent_fmsq_invalid_total_3yr', 'env_penalty_index']))

In [ ]:
X_train_num = NumImputeNormalized(X_train,normalized_cols)
X_valid_num = NumImputeNormalized(X_valid,normalized_cols)
X_test_num = NumImputeNormalized(X_test,normalized_cols)

In [ ]:
X_train[normalized_cols] = X_train_num[normalized_cols]
X_valid[normalized_cols] = X_valid_num[normalized_cols]
X_test[normalized_cols] = X_test_num[normalized_cols]

# Feature Enginerring

In [ ]:
keep_dir = create_dir(_DICT_DIR,'experiment1')

/content/drive/MyDrive/ThesisFinal/dict/experiment1


In [ ]:
n_features=20
fe = FeatureEngineering(X_train, y_train,n_features)
keep_feas,xgb_features,rf_features = fe.drop_importance(keep_dir,_EXP1_EDA_DIR)

Fit XGBoost and Random forests


The most 20 important features:

xgb: ['main_business_元器件', 'industry_code', 'main_business_医疗器械', 'main_business_器件', 'main_business_环境污染', 'main_business_卫星', 'main_business_光纤', 'main_business_定位', 'main_business_飞机', 'main_business_材料', 'main_business_器材', 'main_business_电子器件', 'main_business_航空器', 'ccc_item_code', 'main_business_器具', 'patent_fmsq_cnt_1yr', 'main_business_光机电', 'main_business_机床', 'main_business_集成电路', 'main_business_环境保护']



rf: ['industry_code', 'es_days', 'regcap', 'security_cnt', 'ipclass2_pt', 'ipclass1_pt', 'mainbusinc_stablerate_3yr', 'mainbusinc', 'mainbusinc_1yr', 'relate_ent_cnt', 'progro_profitrate', 'mainbusinc_2yr', 'mainbusinc_grate_1yr', 'patent_indep_cnt', 'ipclass2_cnt', 'mainbusinc_grate_2yr', 'shares_pt_max_indivual', 'ipclass1_cnt', 'progro_roa', 'patent_appl_total_3yr']


drop_importance:  5088
keep:  39
['main_business_飞机', 'industry_code', 'patent_appl_total_3yr', 'main_business_航空器', 'ccc_item_code', 'progro_profitrate', 'mainbusinc_1yr', '

In [ ]:
keep_feas = ['main_business_元器件', 'ccc_item_code', 'patent_indep_cnt', 'main_business_卫星', 'es_days', 'main_business_机床', 'main_business_集成电路', 'regcap', 'ipclass2_pt', 'mainbusinc_stablerate_3yr', 'main_business_光机电', 'patent_fmsq_cnt_1yr', 'ipclass1_pt', 'mainbusinc_grate_1yr', 'main_business_光纤', 'ipclass2_cnt', 'main_business_医疗器械', 'ipclass1_cnt', 'mainbusinc_2yr', 'main_business_器件', 'main_business_定位', 'main_business_器具', 'industry_code', 'shares_pt_max_indivual', 'main_business_环境保护', 'patent_appl_total_3yr', 'main_business_环境污染', 'relate_ent_cnt', 'mainbusinc_1yr', 'progro_roa', 'main_business_电子器件', 'main_business_器材', 'mainbusinc_grate_2yr', 'main_business_材料', 'progro_profitrate', 'mainbusinc', 'main_business_飞机', 'security_cnt']
#most_feas = list(set(['industry_code', 'es_days', 'security_cnt', 'regcap'] + xgb_features))

In [ ]:
xgb_features = ['main_business_元器件', 'industry_code', 'main_business_医疗器械', 'main_business_器件', 'main_business_环境污染', 'main_business_卫星', 'main_business_光纤', 'main_business_定位', 'main_business_飞机', 'main_business_材料', 'main_business_器材', 'main_business_电子器件', 'main_business_航空器', 'ccc_item_code', 'main_business_器具', 'patent_fmsq_cnt_1yr', 'main_business_光机电', 'main_business_机床', 'main_business_集成电路', 'main_business_环境保护']

In [ ]:
most_feas = list(set(['industry_code', 'es_days', 'security_cnt', 'regcap'] + xgb_features))

In [ ]:
set(most_feas)

{'ccc_item_code',
 'es_days',
 'industry_code',
 'main_business_元器件',
 'main_business_光机电',
 'main_business_光纤',
 'main_business_医疗器械',
 'main_business_卫星',
 'main_business_器件',
 'main_business_器具',
 'main_business_器材',
 'main_business_定位',
 'main_business_机床',
 'main_business_材料',
 'main_business_环境保护',
 'main_business_环境污染',
 'main_business_电子器件',
 'main_business_航空器',
 'main_business_集成电路',
 'main_business_飞机',
 'patent_fmsq_cnt_1yr',
 'regcap',
 'security_cnt'}

In [ ]:
len(['tm_apply_cnt_total_3yr',#patent
 'main_business260',#bs
 'zb_grate_1yr',#ar
 'main_business31',#bs
 'main_business170',#bs
 'patent_pledge_cnt_total_3yr',#patent
 'ccc_standard',#pd
 'regcap',#ri
 'security_cnt',#ar
 'patent_fm_cnt',#patent
 'winning_amt_1yr',#patent
 'patent_wgx_cnt_2yr',#patent
 'main_business210.1',#bs
 'ccc_item_code',#pd
 'ccc_vis_prod_unit',#pd
 'patent_fmzl_sqcnt_total_3yr',#patent
 'project_cnt_total_3yr',#tender
 'security_pt',#ar
 'main_business140',#bs
 'industry_code',#ri
 'patent_fmsq_grate_2yr',,#patent
 'ipclass1_pt',,#patent
 'es_days])#ri

23

# Supervised

In [ ]:
sample_weights = compute_sample_weight(class_weight='balanced',y=y_train['industry'])

## All features

In [ ]:
model_lst = ['lr','xgb','svm','rf','mlp']
problem = 'multiclass'
used_feas = X_train.columns.tolist()
report_dir = _SUPERVISED_ALL_RESULTS_DIR

supervised_result_all = RunSupervised(model_lst,problem,X_train,y_train,X_valid,y_valid,X_test,y_test,used_feas,report_dir,algorithm='all')

Model Performance


,Model,acc_valid,acc_test,f1_macro_valid,f1_macro_test
0,lr,34.51,40.86,0.26,0.40
1,xgb,61.41,63.82,0.63,0.66
2,svm,31.22,41.41,0.32,0.40
3,rf,40.07,42.20,0.40,0.43
4,mlp,35.07,41.03,0.31,0.41


## XGBoost Important Features

In [ ]:
model_lst = ['lr','xgb','svm','rf','mlp']
problem = 'multiclass'
used_feas = xgb_features
report_dir = _SUPERVISED_XGB_RESULTS_DIR

supervised_result_xgb = RunSupervised(model_lst,problem,X_train,y_train,X_valid,y_valid,X_test,y_test,used_feas,report_dir,algorithm='supervised')

Model Performance


,Model,acc_valid,acc_test,f1_macro_valid,f1_macro_test
0,lr,41.84,39.11,0.39,0.34
1,xgb,56.78,54.65,0.53,0.52
2,svm,54.73,56.78,0.51,0.56
3,rf,54.27,59.71,0.54,0.61
4,mlp,51.50,48.57,0.53,0.49


## Random Forest Important Features

In [ ]:
model_lst = ['lr','xgb','svm','rf','mlp']
problem = 'multiclass'
used_feas = rf_features
report_dir = _SUPERVISED_RF_RESULTS_DIR

supervised_result_rf = RunSupervised(model_lst,problem,X_train,y_train,X_valid,y_valid,X_test,y_test,used_feas,report_dir,algorithm='supervised')

Model Performance


,Model,acc_valid,acc_test,f1_macro_valid,f1_macro_test
0,lr,38.59,37.72,0.32,0.32
1,xgb,52.47,51.09,0.54,0.54
2,svm,48.35,39.92,0.45,0.38
3,rf,46.56,47.28,0.48,0.50
4,mlp,45.36,43.69,0.46,0.45


## Keep Feas

In [ ]:
model_lst = ['lr','xgb','svm','rf','mlp']
problem = 'multiclass'
used_feas = keep_feas
report_dir = _SUPERVISED_COMBINE_RESULTS_DIR

supervised_result_keep_feas = RunSupervised(model_lst,problem,X_train,y_train,X_valid,y_valid,X_test,y_test,used_feas,report_dir,algorithm='supervised')

Model Performance


,Model,acc_valid,acc_test,f1_macro_valid,f1_macro_test
0,lr,47.35,49.44,0.42,0.44
1,xgb,59.80,60.25,0.62,0.62
2,svm,48.90,44.93,0.46,0.44
3,rf,55.22,55.71,0.59,0.59
4,mlp,51.36,50.56,0.54,0.52


## Most Feas


In [ ]:
model_lst = ['lr','xgb','svm','rf','mlp']
problem = 'multiclass'
used_feas = most_feas
report_dir = _SUPERVISED_SMALL_RESULTS_DIR

supervised_result_most_feas = RunSupervised(model_lst,problem,X_train,y_train,X_valid,y_valid,X_test,y_test,used_feas,report_dir,algorithm='supervised')

Model Performance


,Model,acc_valid,acc_test,f1_macro_valid,f1_macro_test
0,lr,45.11,47.64,0.39,0.43
1,xgb,58.88,59.76,0.60,0.61
2,svm,56.66,59.04,0.52,0.58
3,rf,57.74,60.67,0.61,0.63
4,mlp,53.95,54.62,0.57,0.56


In [ ]:
supervised_result_most_feas.mean(axis=0)

acc_valid         54.468
acc_test          56.346
f1_macro_valid     0.538
f1_macro_test      0.562
dtype: float64

## Fine tuning

In [ ]:
%%time
model_lst = ['lr','xgb','svm','rf','mlp']
best_models = []
problem = 'multiclass'
model_dir = _SUPERVISED_FINETUNING
for model in model_lst:
  print(model)
  sub_best = supervised_fintune(model,problem,model_dir,X_train[keep_feas],y_train,X_valid[keep_feas],y_valid,sample_weights)
  best_models.append(sub_best)
  print('-'*70)

lr
Best hyperparameters for lr: {'C': 0.45883177954303905}
Best objective value for lr: 0.43


----------------------------------------------------------------------
xgb
Best hyperparameters for xgb: {'n_estimators': 102, 'max_depth': 4, 'learning_rate': 0.023685736122769963}
Best objective value for xgb: 0.67


----------------------------------------------------------------------
svm
Best hyperparameters for svm: {'C': 5.275045530264931, 'kernel': 'rbf'}
Best objective value for svm: 0.48


----------------------------------------------------------------------
rf
Best hyperparameters for rf: {'n_estimators': 190, 'max_depth': 10}
Best objective value for rf: 0.61


----------------------------------------------------------------------
mlp
Best hyperparameters for mlp: {'hidden_layer_sizes': 228, 'activation': 'tanh', 'solver': 'adam', 'alpha': 0.017079033694072397, 'learning_rate': 'adaptive'}
Best objective value for mlp: 0.59


----------------------------------------------------------------------
CPU times: user 7min 23s, sys: 1min 2s, total: 8min 25s
Wall time: 8min 19s


In [ ]:
def RunBestSupervised(best_models,X_train,X_test,used_feas,y_train,y_test):
  evals = pd.DataFrame(columns = ['Acc','F1 Macro'])

  for i in range(len(best_models)):
    model = best_models[i]
    display(model)
    model.fit(X_train[used_feas],y_train)
    predict = model.predict(X_test[used_feas])
    acc = round(metrics.balanced_accuracy_score(y_test,predict)*100,2)
    f1 = round(metrics.f1_score(y_test,predict,average='macro'),2)
    report = metrics.classification_report(y_test,predict)
    print(report)

    values = {'Acc':acc,'F1 Macro':f1}
    evals = evals.append(values,ignore_index=True)

  return evals


In [ ]:
Supervised_Best = RunBestSupervised(best_models,X_train,X_test,keep_feas,y_train,y_test)
Supervised_Best.index=model_lst
display(Supervised_Best)

LogisticRegression(C=0.45883177954303905, class_weight='balanced', max_iter=500,
                   random_state=42)

              precision    recall  f1-score   support

         0.0       0.76      0.50      0.60       243
         1.0       0.41      0.42      0.42       112
         2.0       0.60      0.54      0.57       118
         3.0       0.50      0.66      0.57       107
         4.0       0.43      0.32      0.36        95
         5.0       0.22      0.62      0.32        26
         6.0       0.16      0.38      0.22        26

    accuracy                           0.49       727
   macro avg       0.44      0.49      0.44       727
weighted avg       0.56      0.49      0.51       727



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.023685736122769963,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=102, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

              precision    recall  f1-score   support

         0.0       0.83      0.89      0.86       243
         1.0       0.63      0.63      0.63       112
         2.0       0.77      0.64      0.70       118
         3.0       0.48      0.73      0.58       107
         4.0       0.84      0.66      0.74        95
         5.0       0.80      0.46      0.59        26
         6.0       0.75      0.12      0.20        26

    accuracy                           0.71       727
   macro avg       0.73      0.59      0.61       727
weighted avg       0.74      0.71      0.71       727



SVC(C=5.275045530264931, class_weight='balanced', probability=True,
    random_state=42)

...

RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=190,
                       random_state=42)

              precision    recall  f1-score   support

         0.0       0.80      0.87      0.84       243
         1.0       0.71      0.47      0.57       112
         2.0       0.75      0.70      0.72       118
         3.0       0.47      0.78      0.58       107
         4.0       0.79      0.61      0.69        95
         5.0       0.60      0.35      0.44        26
         6.0       0.55      0.23      0.32        26

    accuracy                           0.69       727
   macro avg       0.67      0.57      0.59       727
weighted avg       0.71      0.69      0.69       727



MLPClassifier(activation='tanh', alpha=0.017079033694072397,
              hidden_layer_sizes=228, learning_rate='adaptive', max_iter=100,
              random_state=42)

              precision    recall  f1-score   support

         0.0       0.73      0.85      0.78       243
         1.0       0.63      0.66      0.64       112
         2.0       0.72      0.55      0.62       118
         3.0       0.51      0.65      0.57       107
         4.0       0.60      0.44      0.51        95
         5.0       0.60      0.35      0.44        26
         6.0       0.36      0.19      0.25        26

    accuracy                           0.65       727
   macro avg       0.59      0.53      0.55       727
weighted avg       0.65      0.65      0.64       727



,Acc,F1 Macro
lr,49.13,0.44
xgb,59.08,0.61
svm,46.13,0.46
rf,57.32,0.59
mlp,52.77,0.55


# Generative Models

## SMOTE

In [ ]:
data_dir = create_dir('/content/drive/MyDrive/ThesisFinal/experiment1/data','smote_fake')

/content/drive/MyDrive/ThesisFinal/experiment1/data/smote_fake


In [ ]:
%%time
smote_tuner = SMOTETuner(X_train[keep_feas], y_train)
# Resample the dataset using SMOTE and Optuna
best_params = smote_tuner.tune()

# Print the resampled class distribution and the best parameters


{'k_neighbors': 2, 'sampling_strategy': 'not minority'}
CPU times: user 10min 32s, sys: 5.8 s, total: 10min 37s
Wall time: 11min 20s


### Not miniority

In [ ]:
from generative import Smote
k_neighbors = 2
sampling_strategy = 'not minority'
data_dir =data_dir
dataset = 'exp1'
x_train = X_train[keep_feas]
y_train = y_train
x_train_res, y_train_res = Smote(k_neighbors,sampling_strategy,data_dir,dataset,x_train,y_train)

In [ ]:
y_train_res.head()

,industry
0,2.0
1,4.0
2,4.0
3,5.0
4,2.0


In [ ]:
res_data = x_train_res.copy()
res_data['industry'] = y_train_res['industry']

In [ ]:
train_data = X_train[keep_feas].copy()
train_data['industry'] = y_train['industry']

In [ ]:
Smote_X_train = x_train_res
Smote_y_train = y_train_res

In [ ]:
smote_target_eda = EDA(data = Smote_y_train,
                 target=target,
                 output_path = _EXP1_EDA_DIR,
                 logx = False,
                 logy = False)
smote_target_eda.plot_hist(file_name='smote_trainset',
                     subtitle = 'Source: experiment 1 - combine real train set and SMOTE synthetic data set',
                     maintitle = 'Target ')

In [ ]:
model_lst = ['lr','xgb','rf','mlp']
problem = 'multiclass'
used_feas = keep_feas
report_dir = _GEN_SMOTE_RESULT_DIR

smote_supervised_result_keep_feas = RunSupervised(model_lst,problem,Smote_X_train,Smote_y_train,X_valid,y_valid,X_test,y_test,used_feas,report_dir,algorithm='supervised')

Model Performance


,Model,acc_valid,acc_test,f1_macro_valid,f1_macro_test
0,lr,47.66,47.48,0.42,0.42
1,xgb,58.57,60.04,0.59,0.59
2,rf,48.20,54.85,0.47,0.55
3,mlp,52.41,52.81,0.53,0.52


### Fine Tuning

In [ ]:
%%time
model_lst = ['lr','xgb','rf','mlp']
best_models_smote = []
problem = 'multiclass'
model_dir = '/content/drive/MyDrive/ThesisFinal/experiment1/experiment1/smote_fake'
sample_weights = compute_sample_weight(class_weight='balanced',y=Smote_y_train['industry'])
for model in model_lst:
  print(model)
  sub_best = supervised_fintune(model,problem,model_dir,Smote_X_train,Smote_y_train,X_valid[keep_feas],y_valid,sample_weights)
  best_models_smote.append(sub_best)
  print('-'*70)

lr
Best hyperparameters for lr: {'C': 0.8632529491541137}
Best objective value for lr: 0.42


----------------------------------------------------------------------
xgb
Best hyperparameters for xgb: {'n_estimators': 177, 'max_depth': 4, 'learning_rate': 0.013425891779226706}
Best objective value for xgb: 0.66


----------------------------------------------------------------------
rf
Best hyperparameters for rf: {'n_estimators': 103, 'max_depth': 10}
Best objective value for rf: 0.59


----------------------------------------------------------------------
mlp
Best hyperparameters for mlp: {'hidden_layer_sizes': 154, 'activation': 'tanh', 'solver': 'adam', 'alpha': 0.013837063618430897, 'learning_rate': 'constant'}
Best objective value for mlp: 0.57


----------------------------------------------------------------------
CPU times: user 7min 58s, sys: 3min 12s, total: 11min 10s
Wall time: 10min 13s


In [ ]:
%%time
Supervised_Best_Smote = RunBestSupervised(best_models_smote,Smote_X_train,X_test,keep_feas,Smote_y_train,y_test)
Supervised_Best_Smote.index=model_lst
display(Supervised_Best_Smote)

LogisticRegression(C=0.8632529491541137, class_weight='balanced', max_iter=500,
                   random_state=42)

              precision    recall  f1-score   support

         0.0       0.75      0.38      0.51       243
         1.0       0.40      0.42      0.41       112
         2.0       0.55      0.56      0.55       118
         3.0       0.50      0.66      0.57       107
         4.0       0.35      0.34      0.34        95
         5.0       0.22      0.62      0.32        26
         6.0       0.15      0.35      0.21        26

    accuracy                           0.46       727
   macro avg       0.42      0.47      0.42       727
weighted avg       0.53      0.46      0.47       727



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.013425891779226706,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=177, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

              precision    recall  f1-score   support

         0.0       0.89      0.68      0.77       243
         1.0       0.70      0.52      0.59       112
         2.0       0.58      0.84      0.68       118
         3.0       0.48      0.79      0.60       107
         4.0       0.85      0.71      0.77        95
         5.0       0.48      0.54      0.51        26
         6.0       0.67      0.08      0.14        26

    accuracy                           0.67       727
   macro avg       0.66      0.59      0.58       727
weighted avg       0.72      0.67      0.67       727



RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=103,
                       random_state=42)

              precision    recall  f1-score   support

         0.0       0.83      0.75      0.79       243
         1.0       0.67      0.50      0.57       112
         2.0       0.62      0.75      0.68       118
         3.0       0.49      0.70      0.58       107
         4.0       0.74      0.65      0.69        95
         5.0       0.36      0.50      0.42        26
         6.0       0.62      0.19      0.29        26

    accuracy                           0.66       727
   macro avg       0.62      0.58      0.57       727
weighted avg       0.69      0.66      0.66       727



MLPClassifier(activation='tanh', alpha=0.013837063618430897,
              hidden_layer_sizes=154, max_iter=100, random_state=42)

              precision    recall  f1-score   support

         0.0       0.72      0.69      0.70       243
         1.0       0.60      0.55      0.57       112
         2.0       0.54      0.61      0.57       118
         3.0       0.49      0.58      0.53       107
         4.0       0.54      0.49      0.52        95
         5.0       0.37      0.50      0.43        26
         6.0       0.50      0.15      0.24        26

    accuracy                           0.59       727
   macro avg       0.54      0.51      0.51       727
weighted avg       0.59      0.59      0.59       727



,Acc,F1 Macro
lr,47.48,0.42
xgb,59.22,0.58
rf,57.84,0.57
mlp,51.13,0.51


CPU times: user 9.11 s, sys: 4.62 s, total: 13.7 s
Wall time: 8.25 s


### All

In [ ]:
k_neighbors = 2
sampling_strategy = 'all'
data_dir =data_dir
dataset = 'exp1_all'
x_train = X_train[keep_feas]
y_train = y_train
x_train_res, y_train_res = Smote(k_neighbors,sampling_strategy,data_dir,dataset,x_train,y_train)

In [ ]:
Smote_X_train = x_train_res
Smote_y_train = y_train_res

In [ ]:
smote_target_eda = EDA(data = Smote_y_train,
                 target=target,
                 output_path = _EXP1_EDA_DIR,
                 logx = False,
                 logy = False)
smote_target_eda.plot_hist(file_name='smote_trainset_all',
                     subtitle = 'Source: experiment 1 - combine real train set and SMOTE synthetic data set',
                     maintitle = 'Target ')

In [ ]:
_GEN_SMOTE_RESULT_ALL_DIR = create_dir(_GEN_SMOTE_RESULT_DIR,'all')

/content/drive/MyDrive/ThesisFinal/experiment1/results/smote/all


In [ ]:
model_lst = ['lr','xgb','svm','rf','mlp']
problem = 'multiclass'
used_feas = keep_feas
report_dir = _GEN_SMOTE_RESULT_ALL_DIR

smote_supervised_result_keep_feas = RunSupervised(model_lst,problem,Smote_X_train,Smote_y_train,X_valid,y_valid,X_test,y_test,used_feas,report_dir,algorithm='supervised')

Model Performance


,Model,acc_valid,acc_test,f1_macro_valid,f1_macro_test
0,lr,46.07,46.36,0.40,0.41
1,xgb,58.57,58.74,0.57,0.57
2,svm,45.67,43.12,0.45,0.43
3,rf,52.04,55.76,0.51,0.56
4,mlp,48.10,50.37,0.48,0.49


## CTGAN+

In [ ]:
data = X_train[keep_feas].copy()
data['industry'] = y_train['industry']
data.to_csv('/content/drive/MyDrive/ThesisFinal/data/train_data_label.csv',index=False)

In [ ]:
num_exp = 5
dataset = 'industry'
real_path = '/content/drive/MyDrive/ThesisFinal/data/train_data_label.csv'
fake_file_root = _GEN_CTGANPLUS_DIR
fake_file_root = create_dir(fake_file_root,'fake_data')
cat_col =  ['industry_code','ccc_item_code']
num_col = keep_feas
target='industry'

/content/drive/MyDrive/ThesisFinal/experiment1/experiment1/ctgan+/fake_data


In [ ]:
int_cols = [x for x in keep_feas if x.startswith('main_business')]

In [ ]:
synthesizer =  CTABGAN(raw_csv_path = real_path,
                 test_ratio = 0.30,
                 categorical_columns = cat_col + [target],
                 log_columns = [],
                 mixed_columns= {},
                 general_columns = num_col,
                 non_categorical_columns = [],
                 integer_columns = int_cols,
                 problem_type= {"Classification": target})

In [ ]:
%%time
for i in range(num_exp):
    synthesizer.fit()
    syn = synthesizer.generate_samples()
    syn.to_csv(fake_file_root+"/"+ dataset+"_fake_{exp}.csv".format(exp=i), index= False)

100%|██████████| 150/150 [00:27<00:00,  5.48it/s]


Finished training in 27.611369848251343  seconds.


100%|██████████| 150/150 [00:26<00:00,  5.58it/s]


Finished training in 27.097243785858154  seconds.


100%|██████████| 150/150 [00:29<00:00,  5.01it/s]


Finished training in 30.177919149398804  seconds.


100%|██████████| 150/150 [00:28<00:00,  5.27it/s]


Finished training in 28.597448587417603  seconds.


100%|██████████| 150/150 [00:26<00:00,  5.62it/s]


Finished training in 26.89465880393982  seconds.
CPU times: user 2min 16s, sys: 1.26 s, total: 2min 17s
Wall time: 2min 21s


In [ ]:
fake_paths = glob.glob(fake_file_root+"/"+"*")
fakes= []
for i in fake_paths:
  sub_df = pd.read_csv(i)
  fakes.append(sub_df)
ctgan_fakes = pd.concat(fakes)

In [ ]:
ct_target_eda = EDA(data = ctgan_fakes,
                 target=target,
                 output_path = _EXP1_EDA_DIR,
                 logx = False,
                 logy = False)
ct_target_eda.plot_hist(file_name='CTGANPlus_trainset_all',
                     subtitle = 'Source: experiment 1 - CTGANPlus synthetic data set',
                     maintitle = 'Target ')

In [ ]:
ct_X_train = pd.concat([X_train[keep_feas],ctgan_fakes[keep_feas]])
ct_y_train = pd.concat([y_train,ctgan_fakes[target]])

In [ ]:
ct_y_train[target] = ct_y_train[target].fillna(ct_y_train[0])

In [ ]:
ct_y_train.drop(0,axis=1,inplace=True)

### disbalanced train set

In [ ]:
ct_target_eda = EDA(data = ct_y_train,
                 target=target,
                 output_path = _EXP1_EDA_DIR,
                 logx = False,
                 logy = False)
ct_target_eda.plot_hist(file_name='CTGANPlus_trainset_combined',
                     subtitle = 'Source: experiment 1 - train set augmented with CTGANPlus synthetic data set',
                     maintitle = 'Target ')

### Supervised learning and Fine-tuning

In [ ]:
model_lst = ['lr','xgb','svm','rf','mlp']
problem = 'multiclass'
used_feas = keep_feas
report_dir = _GEN_CTGANPLUS_RESULT_DIR

ct_supervised_result_keep_feas = RunSupervised(model_lst,problem,ct_X_train,ct_y_train,X_valid,y_valid,X_test,y_test,used_feas,report_dir,algorithm='supervised')

Model Performance


,Model,acc_valid,acc_test,f1_macro_valid,f1_macro_test
0,lr,32.87,36.61,0.29,0.34
1,xgb,51.09,50.06,0.54,0.52
2,svm,32.86,41.38,0.31,0.40
3,rf,38.90,45.28,0.38,0.46
4,mlp,41.76,43.52,0.43,0.46


In [ ]:
%%time
model_lst = ['lr','xgb','rf','mlp']
best_models_ct = []
problem = 'multiclass'
model_dir = '/content/drive/MyDrive/ThesisFinal/experiment1/experiment1/ctgan+'
sample_weights = compute_sample_weight(class_weight='balanced',y=ct_y_train['industry'])
for model in model_lst:
  print(model)
  sub_best = supervised_fintune(model,problem,model_dir,ct_X_train,ct_y_train,X_valid[keep_feas],y_valid,sample_weights)
  best_models_ct.append(sub_best)
  print('-'*70)

lr
Best hyperparameters for lr: {'C': 0.589540702372328}
Best objective value for lr: 0.3


----------------------------------------------------------------------
xgb
Best hyperparameters for xgb: {'n_estimators': 151, 'max_depth': 5, 'learning_rate': 0.08108862076169897}
Best objective value for xgb: 0.56


----------------------------------------------------------------------
rf
Best hyperparameters for rf: {'n_estimators': 123, 'max_depth': 9}
Best objective value for rf: 0.44


----------------------------------------------------------------------
mlp
Best hyperparameters for mlp: {'hidden_layer_sizes': 236, 'activation': 'tanh', 'solver': 'adam', 'alpha': 0.008121223125050045, 'learning_rate': 'constant'}
Best objective value for mlp: 0.51


----------------------------------------------------------------------
CPU times: user 48min 12s, sys: 24min 25s, total: 1h 12min 37s
Wall time: 47min 2s


In [ ]:
%%time
Supervised_Best_ct = RunBestSupervised(best_models_ct,ct_X_train,X_test,keep_feas,ct_y_train,y_test)
Supervised_Best_ct.index=model_lst
display(Supervised_Best_ct)

LogisticRegression(C=0.589540702372328, class_weight='balanced', max_iter=500,
                   random_state=42)

              precision    recall  f1-score   support

         0.0       0.54      0.47      0.50       243
         1.0       0.47      0.39      0.43       112
         2.0       0.39      0.56      0.46       118
         3.0       0.34      0.31      0.32       107
         4.0       0.52      0.18      0.27        95
         5.0       0.14      0.27      0.18        26
         6.0       0.14      0.38      0.20        26

    accuracy                           0.40       727
   macro avg       0.36      0.37      0.34       727
weighted avg       0.44      0.40      0.40       727



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.08108862076169897,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=151, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

              precision    recall  f1-score   support

         0.0       0.71      0.93      0.80       243
         1.0       0.62      0.66      0.64       112
         2.0       0.81      0.32      0.46       118
         3.0       0.60      0.64      0.62       107
         4.0       0.57      0.67      0.62        95
         5.0       1.00      0.19      0.32        26
         6.0       0.33      0.08      0.12        26

    accuracy                           0.66       727
   macro avg       0.66      0.50      0.51       727
weighted avg       0.67      0.66      0.63       727



RandomForestClassifier(class_weight='balanced', max_depth=9, n_estimators=123,
                       random_state=42)

              precision    recall  f1-score   support

         0.0       0.77      0.81      0.79       243
         1.0       0.63      0.38      0.48       112
         2.0       0.62      0.65      0.64       118
         3.0       0.43      0.85      0.57       107
         4.0       0.71      0.25      0.37        95
         5.0       0.45      0.19      0.27        26
         6.0       0.27      0.23      0.25        26

    accuracy                           0.61       727
   macro avg       0.55      0.48      0.48       727
weighted avg       0.64      0.61      0.59       727



MLPClassifier(activation='tanh', alpha=0.008121223125050045,
              hidden_layer_sizes=236, max_iter=100, random_state=42)

              precision    recall  f1-score   support

         0.0       0.71      0.82      0.76       243
         1.0       0.55      0.64      0.59       112
         2.0       0.60      0.53      0.56       118
         3.0       0.53      0.54      0.53       107
         4.0       0.55      0.48      0.51        95
         5.0       0.75      0.12      0.20        26
         6.0       0.46      0.23      0.31        26

    accuracy                           0.61       727
   macro avg       0.59      0.48      0.50       727
weighted avg       0.61      0.61      0.60       727



,Acc,F1 Macro
lr,36.61,0.34
xgb,50.07,0.51
rf,48.13,0.48
mlp,47.99,0.50


CPU times: user 1min 4s, sys: 41.6 s, total: 1min 46s
Wall time: 1min 7s


In [ ]:
import numpy as np
np.mean([0.86 ,0.67 , 0.73,0.62,0.75,0.62,0.41
         ])

0.6657142857142857

### balanced train set

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
ct_X_res, ct_y_res = rus.fit_resample(ct_X_train, ct_y_train)

In [ ]:
ct_target_eda = EDA(data = ct_y_res,
                 target=target,
                 output_path = _EXP1_EDA_DIR,
                 logx = False,
                 logy = False)
ct_target_eda.plot_hist(file_name='CTGANPlus_trainset_balanced',
                     subtitle = 'Source: experiment 1 - balanced train set augmented with CTGANPlus synthetic data set',
                     maintitle = 'Target ')

### Supervised Learning

In [ ]:
_BALANED_DIR = create_dir(_GEN_CTGANPLUS_RESULT_DIR,'balanced_report')

/content/drive/MyDrive/ThesisFinal/experiment1/results/ctgan+/balanced_report


In [ ]:
model_lst = ['lr','xgb','svm','rf','mlp']
problem = 'multiclass'
used_feas = keep_feas
report_dir = _BALANED_DIR

ct_supervised_result_keep_feas = RunSupervised(model_lst,problem,ct_X_res,ct_y_res,X_valid,y_valid,X_test,y_test,used_feas,report_dir,algorithm='supervised')

Model Performance


,Model,acc_valid,acc_test,f1_macro_valid,f1_macro_test
0,lr,34.08,34.83,0.30,0.33
1,xgb,50.36,49.82,0.48,0.49
2,svm,31.30,35.43,0.31,0.34
3,rf,51.27,48.89,0.51,0.48
4,mlp,42.88,40.49,0.40,0.38


#### Fine-tuning


In [ ]:
_MODEL_DIR = create_dir('/content/drive/MyDrive/ThesisFinal/experiment1/experiment1/ctgan+','balanced')

/content/drive/MyDrive/ThesisFinal/experiment1/experiment1/ctgan+/balanced


In [ ]:
%%time
model_lst = ['lr','xgb','rf','mlp']
best_models_ct = []
problem = 'multiclass'
model_dir = _MODEL_DIR
sample_weights = compute_sample_weight(class_weight='balanced',y=ct_y_res['industry'])
for model in model_lst:
  print(model)
  sub_best = supervised_fintune(model,problem,model_dir,ct_X_res,ct_y_res,X_valid[keep_feas],y_valid,sample_weights)
  best_models_ct.append(sub_best)
  print('-'*70)

lr
Best hyperparameters for lr: {'C': 0.41470014448341835}
Best objective value for lr: 0.31


----------------------------------------------------------------------
xgb
Best hyperparameters for xgb: {'n_estimators': 178, 'max_depth': 10, 'learning_rate': 0.06217929918502659}
Best objective value for xgb: 0.58


----------------------------------------------------------------------
rf
Best hyperparameters for rf: {'n_estimators': 158, 'max_depth': 9}
Best objective value for rf: 0.46


----------------------------------------------------------------------
mlp
Best hyperparameters for mlp: {'hidden_layer_sizes': 245, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.0019916507299507775, 'learning_rate': 'adaptive'}
Best objective value for mlp: 0.47


----------------------------------------------------------------------
CPU times: user 21min 32s, sys: 4min 24s, total: 25min 57s
Wall time: 24min 23s


In [ ]:
%%time
Supervised_Best_ct = RunBestSupervised(best_models_ct,ct_X_res,X_test,keep_feas,ct_y_res,y_test)
Supervised_Best_ct.index=model_lst
display(Supervised_Best_ct)

LogisticRegression(C=0.41470014448341835, class_weight='balanced', max_iter=500,
                   random_state=42)

              precision    recall  f1-score   support

         0.0       0.50      0.43      0.46       243
         1.0       0.54      0.40      0.46       112
         2.0       0.37      0.59      0.46       118
         3.0       0.41      0.33      0.36       107
         4.0       0.24      0.13      0.17        95
         5.0       0.14      0.19      0.16        26
         6.0       0.12      0.35      0.18        26

    accuracy                           0.39       727
   macro avg       0.33      0.35      0.32       727
weighted avg       0.41      0.39      0.39       727



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.06217929918502659,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=178, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

              precision    recall  f1-score   support

         0.0       0.82      0.73      0.77       243
         1.0       0.56      0.51      0.53       112
         2.0       0.60      0.69      0.64       118
         3.0       0.47      0.68      0.56       107
         4.0       0.73      0.39      0.51        95
         5.0       0.42      0.38      0.40        26
         6.0       0.24      0.38      0.29        26

    accuracy                           0.61       727
   macro avg       0.55      0.54      0.53       727
weighted avg       0.64      0.61      0.62       727



RandomForestClassifier(class_weight='balanced', max_depth=9, n_estimators=158,
                       random_state=42)

              precision    recall  f1-score   support

         0.0       0.80      0.65      0.72       243
         1.0       0.71      0.40      0.51       112
         2.0       0.49      0.72      0.58       118
         3.0       0.45      0.83      0.59       107
         4.0       0.78      0.26      0.39        95
         5.0       0.40      0.23      0.29        26
         6.0       0.17      0.31      0.22        26

    accuracy                           0.57       727
   macro avg       0.54      0.49      0.47       727
weighted avg       0.65      0.57      0.57       727



MLPClassifier(alpha=0.0019916507299507775, hidden_layer_sizes=245,
              learning_rate='adaptive', max_iter=100, random_state=42)

              precision    recall  f1-score   support

         0.0       0.71      0.54      0.62       243
         1.0       0.52      0.43      0.47       112
         2.0       0.43      0.62      0.51       118
         3.0       0.42      0.50      0.45       107
         4.0       0.45      0.28      0.35        95
         5.0       0.23      0.35      0.27        26
         6.0       0.19      0.38      0.26        26

    accuracy                           0.48       727
   macro avg       0.42      0.44      0.42       727
weighted avg       0.52      0.48      0.49       727



,Acc,F1 Macro
lr,34.56,0.32
xgb,53.78,0.53
rf,48.71,0.47
mlp,44.30,0.42


CPU times: user 30.9 s, sys: 5.06 s, total: 35.9 s
Wall time: 33 s
